In [ ]:
#import ollama
import joblib
import numpy as np
from sklearn.model_selection import train_test_split
import glob


def read_pickle(path):
    with open(path, 'rb')as f:
        feats = joblib.load(f)

    return feats

def save_pickle(path, feats):
    with open(path, 'wb')as f:
        joblib.dump(feats, f)

def read_pickle(path):
    with open(path, 'rb')as f:
        feats = joblib.load(f)

    return feats

def save_pickle(path, feats):
    with open(path, 'wb')as f:
        joblib.dump(feats, f)


train_root = r"path/to/train"
# 搜寻train文件夹下的所有文件夹；
class_names_p = glob.glob(r"path/to/train/*")
class_names = [i.split("\\")[-1] for i in class_names_p]
data = read_pickle(r"path/to/piying_all.pickle")

#
train_paths ={}
train_paths_lis, train_paths_labels = [], []
print(f"训练集：")
for name in class_names:
    p = r"path/to/train/%s/*.jpg"%name
    img_p = glob.glob(p)
    train_paths[name] = img_p
    train_paths_lis.extend(img_p)
    train_paths_labels.extend([name]*len(img_p))
    print(f"{name}有图片{len(img_p)}张...")


test_paths = {}
test_paths_lis, test_paths_labels = [], []
print(f"测试集：")
for name in class_names:
    p = r"path/to/test/%s/*.jpg"%name
    img_p = glob.glob(p)
    test_paths[name] = img_p
    test_paths_lis.extend(img_p)
    test_paths_labels.extend([name]*len(img_p))
    print(f"{name}有图片{len(img_p)}张...")

print(f"测试集有图片{len(test_paths_lis)}张...")
trainX_toucha, testX_toucha, trainY_toucha, testY_toucha = train_paths_lis, test_paths_lis, train_paths_labels, test_paths_labels

In [ ]:
import os
from dashscope import MultiModalConversation
from openai import OpenAI
import openai
import time
import base64


line_nex = "\n"

def gen_topK_paths(ids):
    topK_paths = [trainX_toucha[i] for i in ids]
    return topK_paths

# 调用本地图片；
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# 调用大模型；
def model(model_name, messages):

    # gemini
    client =  OpenAI(
        api_key="api_key",
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
    response = client.chat.completions.create(model=model_name, messages=messages)
    return response


def gen_icl_examples(idx, topK_ids, un_topK_ids, test_img_path):
    # 图片所附带的简介知识和关键词；
    id_intro_kw_dic = {i["id"]:{"intro": i["comment"], "kw": i["keywords"]} for i in data}

    topK_paths = gen_topK_paths(topK_ids[idx])
    topK_paths_ids = [int(p.split("/")[-1].split(".")[0]) for p in topK_paths]

    un_topK_paths = gen_topK_paths(topK_ids[idx])
    un_topK_paths_ids = [int(p.split("/")[-1].split(".")[0]) for p in un_topK_paths]

    labels = [trainY_toucha[l] for l in topK_ids[idx]]
    un_labels = [trainY_toucha[l] for l in un_topK_ids[idx]]

    # 正向；
    u_content_p1 = [{"type": "text", "text":
                       f"""
                       请学习以下正例图片及其对应的类别标签及“行当类别”知识，
                       排序越靠前的示例图片表明其类别标签与预测图片类别标签越相似：
                       {["行当类别标签：" + i + "；行当类别知识：" + id_intro_kw_dic[topK_paths_ids[idx]]["intro"] + "；" + id_intro_kw_dic[topK_paths_ids[idx]]["kw"] + line_nex for idx, i in enumerate(labels)]}。
                       """}]
    u_content_p2 = [{"type":"image_url", "image_url": {"url": f"data:image/png;base64,{encode_image(i)}"}} for i in topK_paths]
    #print(u_content_p1)

     # 反向；
    u_content_p3 = [{"type": "text", "text":
                       f"""
                       请学习以下与预测图片不相关的反例图片及其对应的类别标签及“行当类别”知识，
                       排序越靠后的示例图片表明其类别标签与预测图片类别标签越不相似：
                       {["行当类别标签：" + i + "：行当类别知识：" + id_intro_kw_dic[un_topK_paths_ids[idx]]["intro"] + "；" + id_intro_kw_dic[un_topK_paths_ids[idx]]["kw"] + line_nex for idx, i in enumerate(un_labels)]}。
                       """}]
    u_content_p4 = [{"type":"image_url", "image_url": {"url": f"data:image/png;base64,{encode_image(i)}"}} for i in un_topK_paths]

    #1. 正例和反例图片中的行当类别知识是皮影头茬行当类别预测的重要依据；
    u_content_p5 = [{"type": "text", "text":
                     """
                     请结合以下知识进行预测：
                     1. "生"是扮演男性角色的一种行当，其中包括老生：主要扮演帝王及儒雅文弱的中老年人。
                     小生：主要扮演年轻英俊的男性角色。
                     武生：主要扮演的是勇猛战将或是绿林英雄。
                     红生：专指勾红色脸谱的老生。
                     娃娃生：剧中的儿童角色等几大类。
                     除去红生和勾脸（在脸上画有脸谱）的武生以外，一般的生行都是素脸的，即扮相都是比较洁净俊美的；
                     2. 行当"旦"是扮演各种不同年龄，不同性格、不同身份的女性角色。
                     旦行分为青衣（正旦）：端庄娴雅的女子。花旦：天真活泼的少女或性格泼辣的少妇。
                     武旦：扮演勇武的女性人物，特别是打出手。刀马旦：擅长武艺的青壮年妇女，不用打出手。
                     老旦：老年妇女。彩旦：滑稽诙谐的喜剧性人物。
                     花衫：熔青衣、花旦、武旦、刀马旦于一炉的全才演员等称为花衫；
                     3. 行当“净”一般都是扮演男性角色。净行可分为正净（大花脸）：地位较高，举止稳重的忠臣良将。
                     副净（二花脸）：俗称架子花脸，大多扮演性格粗豪莽撞的人物。武净（武花脸）：以武打为主的角色。
                     4. 行当“末”多为中、老年男性人物；
                     5. 行当“丑”又叫小花脸、三花脸。包括文丑：伶俐风趣或阴险狡黠的角色。武丑：精明干练而风趣幽默的豪杰义士；

                     基于学习到的图片及对应的行当类别知识，
                     ，判别下面这张图片属于皮影头茬行当里的哪一个类别，从标签列表["生", "旦", "净", "末", "丑"]中选择一个作为答案。
                     输出格式为：["类别"]
                     """}]

    u_content_p6 = [{"type":"image_url", "image_url": {"url": f"data:image/png;base64,{encode_image(test_img_path)}"}}]

    sys_content = u_content_p1 + u_content_p2 + u_content_p3 + u_content_p4 + u_content_p5 + u_content_p6

    return sys_content


def icl(content, model_name):
    messages = [
        {"role": "system",
         "content": [{"type": "text", "text":
                       """
                       假设你是皮影领域资深专家，请从列表中的标签类别["生", "旦", "净", "末", "丑"]，
                       对图片中皮影头茬的行当类别进行预测。
                       """}]},
        {"role": "user",
        "content": content}
    ]

    response = model(model_name=model_name, messages=messages)

    return response


pres = []
k = 9
model_name = "gemini-2.0-flash"
topK_ids = read_pickle(r"path/to/topK_{}.pickle".format(k))
un_topK_ids = read_pickle(r"path/to/un_topK_{}.pickle".format(k))


print(f"*********{model_name}*********")
for idx, test_img_path in enumerate(testX_toucha[:1500]):
    print("===>真实标签为 %s...<==="%testY_toucha[idx])
    print(test_img_path)
    t_s = time.time()

    print("topK标签为:", [trainY_toucha[l] for l in topK_ids[idx]])
    print("un_topK标签为:", [trainY_toucha[l] for l in un_topK_ids[idx]])

    content = gen_icl_examples(idx=idx, topK_ids=topK_ids, un_topK_ids=un_topK_ids,test_img_path=test_img_path)

    try:
        response = icl(content=content, model_name=model_name)
    except:
        response = icl(content=content, model_name=model_name)


    pre = response.choices[0].message.content

    print("===>预测标签为%s...<==="%pre)
    t_e = time.time()
    print("===>第%d个耗时%f秒...<==="%(idx, t_e-t_s))
    pres.append(pre)
    print("----------------------------------")